In [2]:
%run run_model.py --process=1 --rawname=1 --resume=0 --anchoring=1

24-02-19 01:46:14|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train
--Start Training New!
--Model_name is set to tra_lstm2_day_ShortTest!
use /home/mengkjin/Workspace/learndl/scripts/util/../../data/torch_pack/day+rtn11+res11.20231220.pt


24-02-19 01:46:17|MOD:run_model   |: Finish Process [Load Data]! Cost 3.3Secs
24-02-19 01:46:17|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


tra_lstm2_day_ShortTest #0 @20170103 LoadData Cost    2.3Secs


score function of [pearson] calculated and success!
loss function of [pearson] calculated and success!
penalty function of [hidden_orthogonality] calculated and success!
penalty function of [tra_ot_penalty] calculated and success!


FirstBite Ep#  0 : loss  1.00249, train-0.00246, valid 0.00048, max 0.0005, best 0.0005, lr1.0e-07
FirstBite Ep#  5 : loss  0.88438, train 0.12351, valid 0.05509, max 0.0551, best 0.0551, lr3.8e-03
FirstBite Ep# 10 : loss  0.85125, train 0.16161, valid 0.06219, max 0.0622, best 0.0622, lr1.3e-03
FirstBite Ep# 15 : loss  0.80899, train 0.21267, valid 0.08637, max 0.0864, best 0.0864, lr6.3e-04
24-02-19 01:46:34|MOD:run_model   |: tra_lstm2_day_ShortTest #0 @20170103|Round 0 FirstBite Ep# 19 Max Epoch|Train 0.2216 Valid 0.0825 BestVal 0.0881|Cost  0.3Min,  0.7Sec/Ep
tra_lstm2_day_ShortTest #0 @20170704 LoadData Cost    2.1Secs
FirstBite Ep#  0 : loss  0.99649, train 0.00356, valid 0.00404, max 0.0040, best 0.0040, lr1.0e-07
FirstBite Ep#  5 : loss  0.88273, train 0.12482, valid 0.05928, max 0.0593, best 0.0593, lr3.8e-03
FirstBite Ep# 10 : loss  0.84420, train 0.16967, valid 0.06660, max 0.0828, best 0.0828, lr1.3e-03
FirstBite Ep# 15 : loss  0.81683, train 0.20265, valid 0.08681, max 0.

In [1]:
from run_model import RunModel
from copy import deepcopy
import torch
import torch.nn as nn

config , net , batch_data , model_data , metrics , multiloss = RunModel.new_random_input('simple_lstm' , 'day')
optimizer = RunModel.new_optimizer(net)



/home/mengkjin/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


use /home/mengkjin/Workspace/learndl/scripts/util/../../data/torch_pack/day+rtn11+res11.20231220.pt
Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


In [2]:
net2 = deepcopy(net)

In [3]:
#pipeline 1

sd = deepcopy(net.state_dict())
optimizer = RunModel.new_optimizer(net)
optimizer.zero_grad()
if hasattr(net , 'dynamic_data_assign'): net.dynamic_data_assign(batch_data , model_data)
pred , hidden = net(batch_data['x'])
penalty_kwargs = {'net' : net , 'hidden' : hidden , 'label' : batch_data['y']}
metric = RunModel.calculate_metrics('train' , metrics, label=batch_data['y'], pred=pred,
                            weight=batch_data['w'], multiloss=multiloss,net=net,valid_sample=None,
                            penalty_kwargs=penalty_kwargs)
print(metric)
(metric['loss'] + metric['penalty']).backward()
print(net.get_parameter('fc.weight').grad)
clip_value = config.train_params['trainer']['gradient'].get('clip_value')
if clip_value is not None : nn.utils.clip_grad_value_(net.parameters(), clip_value = clip_value)
optimizer.step()
sd_step = deepcopy(net.state_dict())


score function of [pearson] calculated and success!
loss function of [pearson] calculated and success!
{'loss': tensor(0.9808, grad_fn=<ExpBackward0>), 'loss_item': 0.9807661175727844, 'score': 0.01942128874361515, 'penalty': 0.0, 'losses': None}
tensor([[ 0.0140, -0.0108,  0.0078, -0.0284, -0.0481,  0.0078, -0.0258,  0.0035,
         -0.0371, -0.0073,  0.0234,  0.0281,  0.0275,  0.0296,  0.0173, -0.0114,
          0.0221,  0.0060, -0.0328, -0.0356,  0.0031, -0.0325,  0.0141, -0.0161,
          0.0009, -0.0024,  0.0167,  0.0265,  0.0198,  0.0362, -0.0010, -0.0116,
         -0.0077, -0.0318, -0.0098,  0.0167, -0.0049, -0.0375,  0.0130, -0.0538,
         -0.0527,  0.0242,  0.0462,  0.0368,  0.0090,  0.0316, -0.0234,  0.0166,
         -0.0204, -0.0324,  0.0425,  0.0208, -0.0011,  0.0560, -0.0481,  0.0026,
          0.0176,  0.0150, -0.0724, -0.0247, -0.0164,  0.0246,  0.0033, -0.0301]])


In [4]:
#pipeline 2
sd2 = deepcopy(net2.state_dict())
optimizer2 = RunModel.new_optimizer(net2)
if hasattr(net2 , 'dynamic_data_assign'): net2.dynamic_data_assign(batch_data , model_data)
pred , hidden = net2(batch_data['x'])
penalty_kwargs = {'net' : net2 , 'hidden' : hidden , 'label' : batch_data['y']}
metric = RunModel.calculate_metrics('train' , metrics, label=batch_data['y'], pred=pred,
                            weight=batch_data['w'], multiloss=multiloss,net=net2,valid_sample=None,
                            penalty_kwargs=penalty_kwargs)
optimizer2.zero_grad()
print(metric)
(metric['loss'] + metric['penalty']).backward()
print(net2.get_parameter('fc.weight').grad)
clip_value = config.train_params['trainer']['gradient'].get('clip_value')
if clip_value is not None : nn.utils.clip_grad_value_(net2.parameters(), clip_value = clip_value)
optimizer2.step()
sd2_step = deepcopy(net2.state_dict())


{'loss': tensor(0.9808, grad_fn=<ExpBackward0>), 'loss_item': 0.9807661175727844, 'score': 0.01942128874361515, 'penalty': 0.0, 'losses': None}
tensor([[ 0.0140, -0.0108,  0.0078, -0.0284, -0.0481,  0.0078, -0.0258,  0.0035,
         -0.0371, -0.0073,  0.0234,  0.0281,  0.0275,  0.0296,  0.0173, -0.0114,
          0.0221,  0.0060, -0.0328, -0.0356,  0.0031, -0.0325,  0.0141, -0.0161,
          0.0009, -0.0024,  0.0167,  0.0265,  0.0198,  0.0362, -0.0010, -0.0116,
         -0.0077, -0.0318, -0.0098,  0.0167, -0.0049, -0.0375,  0.0130, -0.0538,
         -0.0527,  0.0242,  0.0462,  0.0368,  0.0090,  0.0316, -0.0234,  0.0166,
         -0.0204, -0.0324,  0.0425,  0.0208, -0.0011,  0.0560, -0.0481,  0.0026,
          0.0176,  0.0150, -0.0724, -0.0247, -0.0164,  0.0246,  0.0033, -0.0301]])


In [30]:
for (k,v),(k2,v2) in zip(sd_step.items(),sd2_step.items()):
    print(k , k2 , (v == v2).all())

lstm.weight_ih_l0 lstm.weight_ih_l0 tensor(True)
lstm.weight_hh_l0 lstm.weight_hh_l0 tensor(True)
lstm.bias_ih_l0 lstm.bias_ih_l0 tensor(True)
lstm.bias_hh_l0 lstm.bias_hh_l0 tensor(True)
fc.weight fc.weight tensor(True)
fc.bias fc.bias tensor(True)


In [22]:
class Test():
    @staticmethod
    def aa(x):
        print(x)

    @staticmethod
    def bb(x):
        print(x)

    def trytry(self , y = 1):
        Test.aa(y)




a = Test()
a.trytry(12)

12


In [2]:
class MyClass:
    def static_method1(self):
        print("This is static method 1")
        self.static_method2()

    @classmethod
    def static_method2(cls , x : int = 0) -> None:
        print(f"This is static method 2")
        print(x)
        return None

# 调用静态方法1
a = MyClass()
a.static_method1()
MyClass.static_method2(x=1)


This is static method 1
This is static method 2
0
This is static method 2
1


In [4]:
import numpy as np
import random
pool = np.arange(100)
random.shuffle(pool) # type: ignore

In [8]:
(lambda x=0:1)(3)

1

In [25]:
sd_step

OrderedDict([('lstm.weight_ih_l0',
              tensor([[ 0.0910,  0.0800,  0.0628,  0.0726, -0.1145, -0.1069],
                      [ 0.0649, -0.1171, -0.1229,  0.0789, -0.0136, -0.1013],
                      [-0.0654,  0.0215, -0.0326, -0.0674, -0.0718, -0.0181],
                      ...,
                      [-0.0225, -0.0093, -0.0572,  0.1153,  0.0017,  0.0702],
                      [ 0.1080,  0.1080,  0.1071,  0.0696, -0.0466,  0.0816],
                      [ 0.0576, -0.0698,  0.0820,  0.1018, -0.0175,  0.0566]])),
             ('lstm.weight_hh_l0',
              tensor([[-0.0280, -0.0936,  0.0853,  ..., -0.0813, -0.1271, -0.1084],
                      [-0.0192, -0.0755,  0.0637,  ...,  0.0753,  0.0598,  0.0185],
                      [ 0.0307,  0.0137,  0.0812,  ..., -0.0837, -0.0918, -0.0508],
                      ...,
                      [-0.0289,  0.0206, -0.0827,  ...,  0.1069,  0.0243, -0.1109],
                      [-0.0859,  0.0232, -0.0917,  ..., -0.0174, -0.0

In [ ]:
for (k,v),(k2,v2) in zip(sd.items(),sd2.items()):
    print(k , k2 , (v == v2).all())

In [71]:
optimizer.zero_grad()